## Get Google Drive

In [ ]:
## Get Google Drive
%load_ext autoreload
from google.colab import drive
import os

drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Breath-Data/data/training/'
os.chdir(path)
!ls

In [ ]:
from glob import glob
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Bronchiolitis/*')


In [ ]:
!pip install pydub

#### Create data breath

In [ ]:
import os
import pandas as pd
import random

from pydub import AudioSegment
from pydub import AudioSegment, effects  
from rnnoise_wrapper import RNNoise

denoiser = RNNoise()

print(denoiser)

duration_breath = 0

In [ ]:
def get_audio_segment (filename, source_path, destination_path, start, end, status, i):
    """[summary]
    
    Arguments:
        filename {[type]} -- [filename of the audio file]
        source_path {[type]} -- [source of a audio file]
        destination_path {[type]} -- [destination of a output file]
        start {[type]} -- [description]
        end {[type]} -- [description]
        status {[type]} -- [type of breath]
    """
    
    # Pydub works in milliseconds
    start = start * 1000 
    end = end * 1000
    offset_time = (end - start)

    global duration_breath
    # print(offset_time)
    print(duration_breath)

    if offset_time > duration_breath and offset_time < 4000:
      duration_breath = offset_time
    
    # print(start, end, end_2)
    
    # Get the audio file
    src_Audio = AudioSegment.from_wav(source_path)
    
    # Get the audio offset
    offset_Audio = AudioSegment.from_wav('/content/gdrive/My Drive/Breath-Data/Sine.wav')
    
    #Cut the right part
    output_audio = src_Audio[start:end]

    try:
      if offset_time > 4000:
        raise Exception("offset_time over 4000")

      # random start audio
      rand = random.randint(0, int(4000 - offset_time))
    
    
      #Convert to 5s
      # output_audio = offset_Audio[0:rand] + output_audio[0:offset_time] + offset_Audio[rand+offset_time+1:4000]

      ###########################################################################
      for j in range(int(end - 4000), int(start), 100):
        if j < 0:
          continue
        output_audio = src_Audio[j:j+4000]
        # Define name file
        fname =  filename + "_" + str(i) + "_" + str(j) + "_" + status + '.wav'
        # print(destination_path)

        # normalized sound
        normalizedsound = effects.normalize(output_audio)
        # denoised_audio = denoiser.filter(normalizedsound, sample_rate=8000, voice_prob_threshold=0.0, save_source_sample_rate=True)

        # denoiser.write_wav(destination_path + '/' + fname, denoised_audio)
        normalizedsound.export(destination_path + '/' + fname, 
                            format="wav")  # Exports to a wav file in the current path.
      ###########################################################################
      
      # Define name file
      # fname =  filename + "_" + str(i) + "_" + status + '.wav'
      # print(destination_path)
      # output_audio.export(destination_path + '/' + fname, 
      #                     format="wav")  # Exports to a wav file in the current path.
    except Exception as e:
      print(e)
      pass

In [ ]:
time_normal = 0
time_deep = 0
count_normal = 0
count_deep = 0


def split_by_label(source_path, destination_path, label_path, output_folder):

    # Check the output directory status 

    check_directory(destination_path, output_folder)

    output_path = os.path.join(destination_path, output_folder) + '/'
    # Get all the audio files
    filenames = os.listdir(source_path)
    
    meta_data=[["",""]]
    
    # Go through all the file 
    for filename in filenames:
        
        # take the file name without dot
        filename =  filename.split(".")[0]
        
        #get wav file name path
        wav_path = source_path + filename + ".wav"
        
        #get label filename path
        csv_path = label_path + filename + ".txt"
        
        #read the label file path 
#         label = pd.read_csv(csv_path, delim_whitespace= True)

        if not os.path.isfile(csv_path):
            print("Not found:", filename)
            continue

        # Open the file with read only permit
        label = open(csv_path, "r")
        # use readlines to read all lines in the file
        # The variable "lines" is a list containing all lines in the file
        lines = label.readlines()
        
#         # split the text
#         words = text.split()
        
        # close the file after reading the lines.
        label.close()
        
        print(csv_path)
        
        i = 0
        
        #Normal breath
        for line in lines:
            breath_start = float(line.split()[0])
            breath_end   = float(line.split()[1])
            try:
                breath = line.split()[2]
                if breath == 'strong':
                  breath = 'heavy'
                if breath == 'normla':
                  breath = 'normal'
            except Exception as e:
                print(e)
                continue
            
            time_breath = (breath_end - breath_start)
            if breath == 'normal' and time_breath > 1.73:
              continue
              # global time_normal
              # time_normal += (breath_end - breath_start)
              # global count_normal
              # count_normal += 1

            if breath == 'deep' and time_breath < 2.52:
              continue

              # global time_deep
              # time_deep += (breath_end - breath_start)
              # global count_deep
              # count_deep += 1

            output_by_label = os.path.join(output_path, breath)               
            if not os.path.exists(output_by_label):
                os.makedirs(output_by_label)

            # check pre and nextline label:
            
            try:
                nextline = lines[lines.index(line)+1]
                breath_next = nextline.split()[2]
                if breath_next == 'strong':
                  breath_next = 'heavy'
                if breath_next == 'normla':
                  breath_next = 'normal'
            except Exception as e:
                print(e)
                continue

            try:
                preline = lines[lines.index(line)+1]
                breath_pre = nextline.split()[2]
                if breath_pre == 'strong':
                  breath_pre = 'heavy'
                if breath_pre == 'normla':
                  breath_pre = 'normal'
            except Exception as e:
                print(e)
                continue

            if breath != breath_pre or breath != breath_next:
              continue
            

            #Export the file
            get_audio_segment(filename, wav_path, output_by_label, breath_start, breath_end, breath, i)
            
            i += 1
            
            # Add other label
            

In [ ]:
# Dev set
DEV_INPUT_PATH = "/content/gdrive/My Drive/Breath-Data/raw_data/training/"
DEV_OUTPUT_PATH = "/content/data/"
DEV_LABEL_PATH = "/content/gdrive/My Drive/Breath-Data/raw_data/training/label/"

TRAIN_OUTPUT_FOLDER = "training"
TEST_OUTPUT_FOLDER = "validation"


# Test set
TEST_INPUT_PATH = "/content/gdrive/My Drive/Breath-Data/raw_data/validation/"
TEST_OUTPUT_PATH = "/content/data/"
TEST_LABEL_PATH = "/content/gdrive/My Drive/Breath-Data/raw_data/validation/label/"

CHUNK = 5
OVERLAP = 0.5

LABEL_FOLDER = 'label'
 # Type of breath
BREATH_TYPE = ['normal', 'deep', 'heavy', 'other']

In [ ]:
os.chdir(TEST_LABEL_PATH)
!ls

In [ ]:
def check_directory(origin_path, folder):
    directory = os.path.join(origin_path, folder)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
# Training set
split_by_label(DEV_INPUT_PATH, DEV_OUTPUT_PATH, DEV_LABEL_PATH, TRAIN_OUTPUT_FOLDER)

# Validation set
split_by_label(TEST_INPUT_PATH, TEST_OUTPUT_PATH, TEST_LABEL_PATH, TEST_OUTPUT_FOLDER)


In [ ]:
# Threadhold time for normal and deep

# print(time_normal/count_normal) # 1.72510334493671
# print(time_deep/count_deep) # 2.5210367994858607

In [ ]:
!cp -r "/content/gdrive/My Drive/Breath-Data/data/training/other" "/content/data/training"

In [ ]:
!cp -r "/content/gdrive/My Drive/Breath-Data/data/validation/other" "/content/data/validation"

#### RNNoise filter

In [ ]:
!sudo apt-get install autoconf libtool

In [ ]:
!git clone https://github.com/Desklop/RNNoise_Wrapper

In [ ]:
!cd RNNoise_Wrapper
!ls
!pwd

In [ ]:
path = '/content/gdrive/My Drive/Breath-Data/data/training/RNNoise_Wrapper'
os.chdir(path)
!ls

In [ ]:
!chmod +x ./compile_rnnoise.sh
!./compile_rnnoise.sh

In [ ]:
from rnnoise_wrapper import RNNoise

denoiser = RNNoise()

audio = denoiser.read_wav('/content/data/training/deep/04_female_21_LAnh_10_deep.wav')
denoised_audio = denoiser.filter(audio, sample_rate=8000, voice_prob_threshold=0.0, save_source_sample_rate=True)
denoiser.write_wav('/content/data/training/test_deep_denoised.wav', denoised_audio)

In [ ]:
from glob import glob
list_breath_path = glob('/content/data/training/deep/*')

print(len(list_breath_path))
print(list_breath_path[0])

audio_file = AudioSegment.from_wav(list_breath_path[0])

print(len(audio_file))
print(audio_file)

In [ ]:
from __future__ import print_function
import sys

import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import concatenate
from keras import losses
from keras import regularizers
from keras.constraints import min_max_norm
import h5py

from keras.constraints import Constraint
from keras import backend as K
import numpy as np

#import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.42
#set_session(tf.Session(config=config))


def my_crossentropy(y_true, y_pred):
    return K.mean(2*K.abs(y_true-0.5) * K.binary_crossentropy(y_pred, y_true), axis=-1)

def mymask(y_true):
    return K.minimum(y_true+1., 1.)

def msse(y_true, y_pred):
    return K.mean(mymask(y_true) * K.square(K.sqrt(y_pred) - K.sqrt(y_true)), axis=-1)

def mycost(y_true, y_pred):
    return K.mean(mymask(y_true) * (10*K.square(K.square(K.sqrt(y_pred) - K.sqrt(y_true))) + K.square(K.sqrt(y_pred) - K.sqrt(y_true)) + 0.01*K.binary_crossentropy(y_pred, y_true)), axis=-1)

def my_accuracy(y_true, y_pred):
    return K.mean(2*K.abs(y_true-0.5) * K.equal(y_true, K.round(y_pred)), axis=-1)

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, c=2):
        self.c = c

    def __call__(self, p):
        return K.clip(p, -self.c, self.c)

    def get_config(self):
        return {'name': self.__class__.__name__,
            'c': self.c}

reg = 0.000001
constraint = WeightClip(0.499)

print('Build model...')
main_input = Input(shape=(None, 42), name='main_input')
tmp = Dense(24, activation='tanh', name='input_dense', kernel_constraint=constraint, bias_constraint=constraint)(main_input)
vad_gru = GRU(24, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, name='vad_gru', kernel_regularizer=regularizers.l2(reg), recurrent_regularizer=regularizers.l2(reg), kernel_constraint=constraint, recurrent_constraint=constraint, bias_constraint=constraint)(tmp)
vad_output = Dense(1, activation='sigmoid', name='vad_output', kernel_constraint=constraint, bias_constraint=constraint)(vad_gru)
noise_input = keras.layers.concatenate([tmp, vad_gru, main_input])
noise_gru = GRU(48, activation='relu', recurrent_activation='sigmoid', return_sequences=True, name='noise_gru', kernel_regularizer=regularizers.l2(reg), recurrent_regularizer=regularizers.l2(reg), kernel_constraint=constraint, recurrent_constraint=constraint, bias_constraint=constraint)(noise_input)
denoise_input = keras.layers.concatenate([vad_gru, noise_gru, main_input])

denoise_gru = GRU(96, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, name='denoise_gru', kernel_regularizer=regularizers.l2(reg), recurrent_regularizer=regularizers.l2(reg), kernel_constraint=constraint, recurrent_constraint=constraint, bias_constraint=constraint)(denoise_input)

denoise_output = Dense(22, activation='sigmoid', name='denoise_output', kernel_constraint=constraint, bias_constraint=constraint)(denoise_gru)

model = Model(inputs=main_input, outputs=[denoise_output, vad_output])

model.compile(loss=[mycost, my_crossentropy],
              metrics=[msse],
              optimizer='adam', loss_weights=[10, 0.5])

model.summary()

In [ ]:
!pip install audiosegment

In [ ]:
# Example for plotting the FFT using this function
import matplotlib.pyplot as plt
import numpy as np

seg = audiosegment.from_file(list_breath_path[0])

print(len(seg))
# Just take the first 3 seconds
hist_bins, hist_vals = seg[1:4000].fft()
hist_vals_real_normed = np.abs(hist_vals) / len(hist_vals)
plt.plot(hist_bins / 1000, hist_vals_real_normed)
plt.xlabel("kHz")
plt.ylabel("dB")
plt.show()

In [ ]:
!pip install --upgrade quantecon

In [ ]:
from scipy import linalg
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline
from quantecon import Kalman, LinearStateSpace
from scipy.stats import norm
from scipy.integrate import quad
from numpy.random import multivariate_normal
from scipy.linalg import eigvals

ϵ = 0.1
θ = 10  # Constant value of state x_t
A, C, G, H = 1, 0, 1, 1
ss = LinearStateSpace(A, C, G, H, mu_0=θ)

x_hat_0, Σ_0 = 8, 1
kalman = Kalman(ss, x_hat_0, Σ_0)

T = len(seg)
z = np.empty(T)
x, y = seg[1:4000].fft()
y = y.flatten()


print(len(x))
print(len(y))

for t in range(T):
    # Record the current predicted mean and variance and plot their densities
    m, v = [temp for temp in (kalman.x_hat, kalman.Sigma)]

    f = lambda x: norm.pdf(x, loc=m, scale=np.sqrt(v))
    integral, error = quad(f, θ - ϵ, θ + ϵ)
    z[t] = 1 - integral

    kalman.update(y[t])

fig, ax = plt.subplots(figsize=(9, 7))
ax.set_ylim(0, 1)
ax.set_xlim(0, T)
ax.plot(range(T), z)
ax.fill_between(range(T), np.zeros(T), z, color="blue", alpha=0.2)
plt.show()

#### Respiratory sound database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydub

In [ ]:
from glob import glob
from shutil import copyfile
import pandas as pd
from pydub import AudioSegment

list_label_path = glob('/content/drive/My Drive/Breath-Data/public-data/label/*')
list_breath_path = glob('/content/drive/My Drive/Breath-Data/public-data/*')

In [ ]:
print(len(list_label_path))

In [ ]:
import os
import pandas as pd
import random

from pydub import AudioSegment
from pydub import AudioSegment, effects  
# from rnnoise_wrapper import RNNoise

# denoiser = RNNoise()

# print(denoiser)

duration_breath = 0

In [ ]:
def get_audio_segment (filename, source_path, destination_path, start, end, status, i):
    """[summary]
    
    Arguments:
        filename {[type]} -- [filename of the audio file]
        source_path {[type]} -- [source of a audio file]
        destination_path {[type]} -- [destination of a output file]
        start {[type]} -- [description]
        end {[type]} -- [description]
        status {[type]} -- [type of breath]
    """
    
    # Pydub works in milliseconds
    start = start * 1000 
    end = end * 1000
    offset_time = (end - start)

    global duration_breath
    # print(offset_time)
    print(duration_breath)

    if offset_time > duration_breath and offset_time < 4000:
      duration_breath = offset_time
    
    # print(start, end, end_2)
    
    # Get the audio file
    src_Audio = AudioSegment.from_wav(source_path)
    
    # Get the audio offset
    offset_Audio = AudioSegment.from_wav('/content/drive/My Drive/Breath-Data/Sine.wav')
    
    #Cut the right part
    output_audio = src_Audio[start:end]

    try:
      if offset_time > 4000:
        raise Exception("offset_time over 4000")

      # random start audio
      rand = random.randint(0, int(4000 - offset_time))
    
    
      #Convert to 5s
      # output_audio = offset_Audio[0:rand] + output_audio[0:offset_time] + offset_Audio[rand+offset_time+1:4000]

      ###########################################################################
      for j in range(int(end - 4000), int(start), 100):
        if j < 0:
          continue
        output_audio = src_Audio[j:j+4000]
        # Define name file
        fname =  filename + "_" + str(i) + "_" + str(j) + "_" + status + '.wav'
        # print(destination_path)

        # normalized sound
        normalizedsound = effects.normalize(output_audio)
        # denoised_audio = denoiser.filter(normalizedsound, sample_rate=8000, voice_prob_threshold=0.0, save_source_sample_rate=True)

        # denoiser.write_wav(destination_path + '/' + fname, denoised_audio)
        normalizedsound.export(destination_path + '/' + fname, 
                            format="wav")  # Exports to a wav file in the current path.
        # break
      ###########################################################################
      
      # Define name file
      # fname =  filename + "_" + str(i) + "_" + status + '.wav'
      # print(destination_path)
      # output_audio.export(destination_path + '/' + fname, 
      #                     format="wav")  # Exports to a wav file in the current path.
    except Exception as e:
      print(e)
      pass

In [ ]:
time_normal = 0
time_deep = 0
count_normal = 0
count_deep = 0

count_crackle = 0
count_wheeze = 0
count_all_one = 0
count_all_zero = 0


def split_by_label(source_path, destination_path, label_path, output_folder):

    # Check the output directory status 

    check_directory(destination_path, output_folder)

    output_path = os.path.join(destination_path, output_folder) + '/'
    # Get all the audio files
    filenames = os.listdir(source_path)
    
    meta_data=[["",""]]
    
    # Go through all the file 
    for filename in filenames:
        
        # take the file name without dot
        filename =  filename.split(".")[0]
        
        #get wav file name path
        wav_path = source_path + filename + ".wav"
        
        #get label filename path
        csv_path = label_path + filename + ".txt"
        
        #read the label file path 
#         label = pd.read_csv(csv_path, delim_whitespace= True)

        if not os.path.isfile(csv_path):
            print("Not found:", filename)
            continue

        # Open the file with read only permit
        label = open(csv_path, "r")
        # use readlines to read all lines in the file
        # The variable "lines" is a list containing all lines in the file
        lines = label.readlines()
        
#         # split the text
#         words = text.split()
        
        # close the file after reading the lines.
        label.close()
        
        print(csv_path)

        # break
        
        i = 0
        breath = ''
        
        #Normal breath
        for line in lines:
            breath_start = float(line.split()[0])
            breath_end   = float(line.split()[1])

            # print(breath_start)
            # print(breath_end)
            breath_label_crackle = float(line.split()[2])
            breath_label_wheeze = float(line.split()[3])
            # print(int(breath_label_crackle))
            # print(int(breath_label_wheeze))

            if (int(breath_label_crackle) == 1 and int(breath_label_wheeze) == 1):
              global count_all_one
              count_all_one += 1

              #######################
              breath = 'crackle'
              output_by_label = os.path.join(output_path, breath)               
              if not os.path.exists(output_by_label):
                  os.makedirs(output_by_label)
              
              # #Export the file
              get_audio_segment(filename, wav_path, output_by_label, breath_start, breath_end, breath, i)
              
              i += 1

              ################
              breath = 'wheeze'
              output_by_label = os.path.join(output_path, breath)               
              if not os.path.exists(output_by_label):
                  os.makedirs(output_by_label)
              
              # #Export the file
              get_audio_segment(filename, wav_path, output_by_label, breath_start, breath_end, breath, i)
              
              i += 1

              ##########
              continue
            elif (int(breath_label_crackle) == 1):
              global count_crackle
              count_crackle += 1
              breath = 'crackle'
            elif (int(breath_label_wheeze) == 1):
              global count_wheeze
              count_wheeze += 1
              breath = 'wheeze'
            else:
              global count_all_zero
              count_all_zero += 1
              breath = 'normal'



            # break
        # break
            # try:
            #     breath = line.split()[2]
            #     if breath == 'strong':
            #       breath = 'heavy'
            #     if breath == 'normla':
            #       breath = 'normal'
            # except Exception as e:
            #     print(e)
            #     continue
            
            # time_breath = (breath_end - breath_start)
            # if breath == 'normal' and time_breath > 1.73:
            #   continue
            #   # global time_normal
            #   # time_normal += (breath_end - breath_start)
            #   # global count_normal
            #   # count_normal += 1

            # if breath == 'deep' and time_breath < 2.52:
            #   continue

            #   # global time_deep
            #   # time_deep += (breath_end - breath_start)
            #   # global count_deep
            #   # count_deep += 1

            output_by_label = os.path.join(output_path, breath)               
            if not os.path.exists(output_by_label):
                os.makedirs(output_by_label)
            
            # # #Export the file
            # get_audio_segment(filename, wav_path, output_by_label, breath_start, breath_end, breath, i)
            
            # i += 1

            # # check pre and nextline label:
            
            # try:
            #     nextline = lines[lines.index(line)+1]
            #     breath_next = nextline.split()[2]
            #     if breath_next == 'strong':
            #       breath_next = 'heavy'
            #     if breath_next == 'normla':
            #       breath_next = 'normal'
            # except Exception as e:
            #     print(e)
            #     continue

            # try:
            #     preline = lines[lines.index(line)+1]
            #     breath_pre = nextline.split()[2]
            #     if breath_pre == 'strong':
            #       breath_pre = 'heavy'
            #     if breath_pre == 'normla':
            #       breath_pre = 'normal'
            # except Exception as e:
            #     print(e)
            #     continue

            # if breath != breath_pre or breath != breath_next:
            #   continue
            

            # #Export the file
            get_audio_segment(filename, wav_path, output_by_label, breath_start, breath_end, breath, i)
            
            i += 1
            
            # # Add other label
            

In [ ]:
# Dev set
DEV_INPUT_PATH = "/content/drive/My Drive/Breath-Data/public-data/"
DEV_OUTPUT_PATH = "/content/data/public/"
DEV_LABEL_PATH = "/content/drive/My Drive/Breath-Data/public-data/label/"

TRAIN_OUTPUT_FOLDER = "training"
TEST_OUTPUT_FOLDER = "validation"


# Test set
TEST_INPUT_PATH = "/content/gdrive/My Drive/Breath-Data/raw_data/validation/"
TEST_OUTPUT_PATH = "/content/data/public/"
TEST_LABEL_PATH = "/content/gdrive/My Drive/Breath-Data/raw_data/validation/label/"

CHUNK = 5
OVERLAP = 0.5

LABEL_FOLDER = 'label'
 # Type of breath
BREATH_TYPE = ['normal', 'deep', 'heavy', 'other']

In [ ]:
def check_directory(origin_path, folder):
    directory = os.path.join(origin_path, folder)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
# Training set
split_by_label(DEV_INPUT_PATH, DEV_OUTPUT_PATH, DEV_LABEL_PATH, TRAIN_OUTPUT_FOLDER)

print(count_crackle)
print(count_wheeze)
print(count_all_one)
print(count_all_zero)

# Validation set
# split_by_label(TEST_INPUT_PATH, TEST_OUTPUT_PATH, TEST_LABEL_PATH, TEST_OUTPUT_FOLDER)


In [ ]:
list_file = glob('/content/drive/My Drive/Breath-Data/public-data/label/*')

In [ ]:
print(len(list_file))
list_name = []
for name in list_file:
  # print(name.split('/')[-1].split('_')[0])
  if name.split('/')[-1].split('_')[0] not in list_name:
    list_name.append(name.split('/')[-1].split('_')[0])
  # break

In [ ]:
print(len(list_name))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from glob import glob
from shutil import copyfile

import pandas as pd
from pydub import AudioSegment

list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/public-data/*')
df = pd.read_csv('/content/gdrive/My Drive/Breath-Data/patient_diagnosis.csv')

print("df - number = ", df.number[0])
print("df - label = ", df.label[0])
print("df - len = ", len(df))

print('len list = ', len(list_breath_path))

for path in list_breath_path:
  print(path)
  print('file name = ', path.split('/')[-1])
  # print('number file name = ', path.split('/')[-1].split('_')[0])
  label = None

  for i in range(0, len(df)):
    # print(df.number[i])
    if int(df.number[i]) == int(path.split('/')[-1].split('_')[0]):
      print('label = ', df.label[i])
      label = df.label[i]
      break

  # output_dist = '/content/gdrive/My Drive/Breath-Data/data-public/' + label
  output_dist = '/content/data-public/' + label
  if not os.path.exists(output_dist):
    os.makedirs(output_dist)
  

  audio_file = AudioSegment.from_wav(path)

  if label == 'COPD':
    for i in range(1, int(len(audio_file)/4000)):
      # print(len(audio_file[4000*(i-1):4000*i]))
      output_dist_1 = output_dist + '/' + str(i) + '_' +  path.split('/')[-1]
      audio_file[4000*(i-1):4000*i].export(output_dist_1, format="wav")
    #   break

    # break
  else:
    for i in range(1, int(len(audio_file)/100)):
      # print(len(audio_file[100*(i-1):100*(i-1)+4000]))
      output_dist_1 = output_dist + '/' + str(i) + '_' +  path.split('/')[-1]
      audio_file[100*(i-1):100*(i-1)+4000].export(output_dist_1, format="wav")
    #   break
    # break
  # break

In [ ]:
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Bronchiolitis/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Bronchiectasis/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/LRTI/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Asthma/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/URTI/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Pneumonia/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Healthy/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/COPD/*')
print(len(list_breath_path))


In [ ]:
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Bronchiolitis/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Bronchiectasis/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/LRTI/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Asthma/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/URTI/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Pneumonia/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/Healthy/*')
print(len(list_breath_path))
list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/COPD/*')
print(len(list_breath_path))

In [ ]:
list_breath_path = glob('/content/data/public/training/crackle/*')
print(len(list_breath_path))
list_breath_path = glob('/content/data/public/training/normal/*')
print(len(list_breath_path))
list_breath_path = glob('/content/data/public/training/wheeze/*')
print(len(list_breath_path))
list_breath_path = glob('/content/data/public/training/abnormal/*')
print(len(list_breath_path))

# 1304 - crackle
# 2491 - normal
# 605 - wheeze

#### Copy file

In [ ]:
print(len(list_breath_path))
print(int(len(list_breath_path)*20/100))
print(list_breath_path[0].split('/')[-2])

In [ ]:
import shutil

# Bronchiolitis
# Bronchiectasis
# LRTI
# Asthma
# URTI
# Pneumonia
# Healthy
# COPD

# list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/COPD/*')
list_breath_path = glob('/content/data/public/training/normal/*')

count = 0
for path in list_breath_path:
  count += 1
  if count == int(len(list_breath_path)*20/100):
    break
  
  output_dist = '/content/data-public/data-validation/' + path.split('/')[-2] + '/'
  if not os.path.exists(output_dist):
    os.makedirs(output_dist)
  output_dist = output_dist + path.split('/')[-1]

  shutil.move(path, output_dist)

In [ ]:
list_breath_path = glob('/content/data-public/data-validation/crackle/*')
print(len(list_breath_path))
list_breath_path = glob('/content/data-public/data-validation/normal/*')
print(len(list_breath_path))
list_breath_path = glob('/content/data-public/data-validation/wheeze/*')
print(len(list_breath_path))
list_breath_path = glob('/content/data-public/data-validation/abnormal/*')
print(len(list_breath_path))

In [ ]:
import shutil

# Bronchiolitis
# Bronchiectasis
# LRTI
# Asthma
# URTI
# Pneumonia
# Healthy
# COPD

# list_breath_path = glob('/content/gdrive/My Drive/Breath-Data/data-public/COPD/*')
list_breath_path = glob('/content/data/public/training/crackle/*')

count = 0
for path in list_breath_path:
  count += 1
  if count == int(len(list_breath_path)*100/100):
    break
  
  output_dist = '/content/data/public/training/abnormal/'
  if not os.path.exists(output_dist):
    os.makedirs(output_dist)
  output_dist = output_dist + path.split('/')[-1]

  shutil.copy(path, output_dist)

In [ ]:
from glob import glob
heavy_path = glob('/content/gdrive/My Drive/Breath-Data/data/validation/heavy/*')
print(len(heavy_path))
deep_path = glob('/content/gdrive/My Drive/Breath-Data/data/training/deep/*')

In [ ]:
list_user = []
count  = 0
dictCount = {}
for path in heavy_path:
  user = path.split('/')[-1].split('_')[0]
  # print(user)
  if user not in list_user:
    list_user.append(user)
    dictCount.setdefault(user, 0)
  
  count += 1
  # print(list_user)
  # if count == 50:
  #   break

In [ ]:
print(len(list_user))
print(list_user)
print(dictCount['01'])

In [ ]:
from glob import glob
dst_path = '/content/gdrive/My Drive/Breath-Data/data/validation/breath/'
src_path = glob('/content/gdrive/My Drive/Breath-Data/data/validation/deep/*')
from shutil import copyfile
count = 0


for path in src_path:
  user = path.split('/')[-1].split('_')[0]
  file_name = path.split('/')[-1]
  dst = dst_path + file_name
  
  # if count <= 112:
  #   count += 1
  if dictCount[user] <= 38:
    dictCount[user] += 1
    copyfile(path, dst)
  # else:
  #   break


  # count += 1
  # break

In [ ]:
from glob import glob
breath_path_new = glob('/content/gdrive/My Drive/Breath-Data/data/validation/breath/*')

print(len(breath_path_new))

## Import Lib

In [ ]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import json
import pickle
from scipy.io import wavfile

# os.environ["CUDA_DEVICE_ORDER"]="0"
import tensorflow as tf

with tf.device('/device:GPU:0'):

    import tensorflow.keras as keras
    import tensorflow as tf

# Allow memory growth for the GPU
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices, True)

# from tensorflow.keras.utils import multi_gpu_model
# from keras.backend.tensorflow_backend import set_session
    import librosa
    from sklearn.metrics import classification_report, confusion_matrix
    from tensorflow.keras.callbacks import ModelCheckpoint
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, Conv1D
    from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Permute, Reshape, TimeDistributed
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Flatten
    from tensorflow.compat.v1.keras.layers import CuDNNLSTM as CuLSTM
    from tensorflow.compat.v1.keras.layers import Input, Dense, Lambda, Layer
    from tensorflow.keras.layers import add
    from tensorflow.keras.layers import Input
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import BatchNormalization
    from tensorflow.keras.layers.experimental import preprocessing
    from tensorflow.keras import regularizers
    from tensorflow.keras.layers import LeakyReLU

    normalize = preprocessing.Normalization()

## Init Data Generator

In [ ]:
!pip install pydub

In [ ]:
import tensorflow as tf
import h5py
import numpy as np
import pandas as pd
from tensorflow.python.keras import backend as K
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Dense, Activation, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
import sklearn
from librosa.util import normalize
from sklearn import preprocessing

from pydub import AudioSegment, effects 
def normalize_manual(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

def normalize_fixed(x, current_range =[[0, 100]], normed_range=[[0, 1]]):
    current_min, current_max = tf.expand_dims(current_range[:, 0], 1), tf.expand_dims(current_range[:, 1], 1)
    normed_min, normed_max = tf.expand_dims(normed_range[:, 0], 1), tf.expand_dims(normed_range[:, 1], 1)
    x_normed = (x - current_min) / (current_max - current_min)
    x_normed = x_normed * (normed_max - normed_min) + normed_min
    return x_normed

def despike(yi, th=1.e-8):
    '''Remove spike from array yi, the spike area is where the difference between the neigboring points is higher than th.'''
    y = np.copy(yi) # use y = y1 if it is OK to modify input array
    n = len(y)
    x = np.arange(n)
    c = np.argmax(y)
    d = abs(np.diff(y))
    try:
        l = c - 1 - np.where(d[c-1::-1]<th)[0][0]
        r = c + np.where(d[c:]<th)[0][0] + 1
    except: # no spike, return unaltered array
        return y
    # for fit, use area twice wider then the spike
    if (r-l) <= 3:
        l -= 1
        r += 1
    s = int(round((r-l)/2.))
    lx = l - s
    rx = r + s
    # make a gap at spike area
    xgapped = np.concatenate((x[lx:l],x[r:rx]))
    ygapped = np.concatenate((y[lx:l],y[r:rx]))
    # quadratic fit of the gapped array
    z = np.polyfit(xgapped,ygapped,2)
    p = np.poly1d(z)
    y[l:r] = p(x[l:r])
    return y

class BreathDataTrainingGenerator(tf.keras.utils.Sequence):#tensorflow.python.keras.utils.data_utils --- tf.keras.utils.Sequence
    'Generates data for Keras'
    def __init__(self, directory, 
                    list_labels=['normal', 'deep', 'heavy', 'other'], 
                    batch_size=32,
                    dim=None,
                    classes=None, 
                    shuffle=True):
        'Initialization'
        self.directory = directory
        self.list_labels = list_labels
        self.dim = dim
        self.__flow_from_directory(self.directory)
        self.batch_size = batch_size
        self.classes = len(self.list_labels)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.wavs) / self.batch_size))

    def __getitem__(self, index):
        # print("In get Item!!")
        # 'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        rawX = [self.wavs[k] for k in indexes]
        rawY = [self.labels[k] for k in indexes]

        # Generate data
        X, Y = self.__feature_extraction(rawX, rawY)
        # print("TADDDDDDDD", len([X, Y, Y]), " - ", len([X, Y, Y][0]), " - ", len([X, Y, Y][0][0]))
        # print("Done getting data")
        return X, Y

    def __flow_from_directory(self, directory):
        self.wavs = []
        self.labels = []
        for dir in os.listdir(directory):
            sub_dir = os.path.join(directory, dir)
            if os.path.isdir(sub_dir) and dir in self.list_labels:
                print(sub_dir)
                label = self.list_labels.index(dir)
                try:
                    for file in os.listdir(sub_dir):
                        if file == '22_male_21_VHung_41_225121_deep.wav':
                            continue
                        
                        self.wavs.append(os.path.join(sub_dir, file))
                        self.labels.append(label)
                except Exception as e:
                    print(e)
                    print(file)
                    pass


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.wavs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __feature_extraction(self, list_wav, list_label):
        # print("Go to feature extraction!!!")
        'Generates data containing batch_size samples'
        X = []
        Y = []
        for i in range(self.batch_size):
            rate, data = wavfile.read(list_wav[i]) #bug in here

            # data, rate = readCoughData(file=list_wav[i])


            # resampledData = resample(originalData=data, origSampFreq=rate, targetSampFreq=16000)
            # normalizedData = normalizeSound(resampledData, axis=0)
            # S = calculateMelSpectogram(normalizedData=normalizedData, hop_length=512, win_length=1024, sr=16000)
            # plotSound(soundData=normalizedData, sr=8000,x_axis_string='time')
            # plotMelSpectogram(S, sr=8000, ref=np.max)


            # data = effects.normalize(data)
            # print("End")
            # data = despike(data)

            data = np.array(data, dtype=np.float32)
            data *= 1./32768
            # feature = librosa.feature.melspectrogram(y=data, sr=rate, n_fft=2048, hop_length=512, power=2.0)
            # feature = librosa.feature.rmse(data+ 0.0001)
            # feature = librosa.feature.spectral_rolloff(y=data, sr=rate)[0]
            feature = librosa.feature.mfcc(y=data, sr=rate, 
                                           n_mfcc=40, fmin=0, fmax=8000,
                                           n_fft=int(16*64), hop_length=int(16*32), power=2.0)

            # print(S.shape) # (128, 251)
            # feature = normalize(feature)
            feature = np.resize(feature, self.dim)

            # mellog = np.log(feature + 1e-9)
            # feature = librosa.util.normalize(mellog)
            # feature= sklearn.preprocessing.normalize(feature)

            # normalize data
            # feature = normalize(feature)

            category_label =  to_categorical(list_label[i], num_classes= len(self.list_labels) )
            X.append(feature)
            Y.append(category_label)
        
        # print(X)
        # X = normalize_fixed(X)
        # X -= np.mean(X, keepdims=True)
        # X = (X- np.min(X, 0)) / (np.max(X, 0) + 0.0001) 
        # X /= (np.std(X, keepdims=True) + tf.keras.backend.epsilon())
        X = np.array(X, dtype=np.float32)
        mean = np.mean(X, axis=0)
        std = np.std(X, axis=0)

        X = (X - mean)/std

        # print(len(X[0]))
        # print(len(X[0][0]))

        # min_max_scaler = preprocessing.MinMaxScaler()
        
        Y = np.array(Y, dtype=int)
        return X, Y




### Data Generator Multitask

In [ ]:
import sklearn
from librosa.util import normalize
from sklearn import preprocessing

from pydub import AudioSegment, effects 

class BreathDataTrainingGeneratorMulti(tf.keras.utils.Sequence):#tensorflow.python.keras.utils.data_utils --- tf.keras.utils.Sequence
    'Generates data for Keras'
    def __init__(self, directory, 
                    list_labels=['normal', 'deep', 'heavy', 'other'], 
                    batch_size=32,
                    dim=None,
                    classes=None, 
                    shuffle=True):
        'Initialization'
        self.directory = directory
        self.list_labels = list_labels
        self.dim = dim
        self.__flow_from_directory(self.directory)
        self.batch_size = batch_size
        self.classes = len(self.list_labels)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.wavs) / self.batch_size))

    def __getitem__(self, index):
        # print("In get Item!!")
        # 'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        rawX = [self.wavs[k] for k in indexes]
        rawY = [self.labels[k] for k in indexes]

        # Generate data
        X, Y, Y1 = self.__feature_extraction(rawX, rawY)
        # print("TADDDDDDDD", len([X, Y, Y]), " - ", len([X, Y, Y][0]), " - ", len([X, Y, Y][0][0]))
        # print("Done getting data")
        return X, [Y, Y1]

    def __flow_from_directory(self, directory):
        self.wavs = []
        self.labels = []
        for dir in os.listdir(directory):
            sub_dir = os.path.join(directory, dir)
            if os.path.isdir(sub_dir) and dir in self.list_labels:
                print(sub_dir)
                label = self.list_labels.index(dir)
                try:
                    for file in os.listdir(sub_dir):
                        if file == '22_male_21_VHung_41_225121_deep.wav':
                            continue
                        
                        self.wavs.append(os.path.join(sub_dir, file))
                        self.labels.append(label)
                except Exception as e:
                    print(e)
                    print(file)
                    pass


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.wavs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __feature_extraction(self, list_wav, list_label):
        # print("Go to feature extraction!!!")
        'Generates data containing batch_size samples'
        X = []
        Y = []
        Y1 = []
        for i in range(self.batch_size):
            rate, data = wavfile.read(list_wav[i]) #bug in here

            # data, rate = readCoughData(file=list_wav[i])


            # resampledData = resample(originalData=data, origSampFreq=rate, targetSampFreq=16000)
            # normalizedData = normalizeSound(resampledData, axis=0)
            # S = calculateMelSpectogram(normalizedData=normalizedData, hop_length=512, win_length=1024, sr=16000)
            # plotSound(soundData=normalizedData, sr=8000,x_axis_string='time')
            # plotMelSpectogram(S, sr=8000, ref=np.max)


            # data = effects.normalize(data)
            # print("End")
            # data = despike(data)

            data = np.array(data, dtype=np.float32)
            data *= 1./32768
            # feature = librosa.feature.melspectrogram(y=data, sr=rate, n_fft=2048, hop_length=512, power=2.0)
            # feature = librosa.feature.rmse(data+ 0.0001)
            # feature = librosa.feature.spectral_rolloff(y=data, sr=rate)[0]
            feature = librosa.feature.mfcc(y=data, sr=rate, 
                                           n_mfcc=40, fmin=0, fmax=8000,
                                           n_fft=int(16*64), hop_length=int(16*32), power=2.0)

            # print(S.shape) # (128, 251)
            # feature = normalize(feature)
            feature = np.resize(feature, self.dim)

            # mellog = np.log(feature + 1e-9)
            # feature = librosa.util.normalize(mellog)
            # feature= sklearn.preprocessing.normalize(feature)

            # normalize data
            # feature = normalize(feature)

            category_label =  to_categorical(list_label[i], num_classes= len(self.list_labels) )
            if list_label[i] == 1:
                category_label1 =  to_categorical(0, num_classes= 2) # non-breath
            else:
                category_label1 =  to_categorical(1, num_classes= 2) # breath
            X.append(feature)
            Y.append(category_label)
            Y1.append(category_label1)
        
        # print(X)
        # X = normalize_fixed(X)
        # X -= np.mean(X, keepdims=True)
        # X = (X- np.min(X, 0)) / (np.max(X, 0) + 0.0001) 
        # X /= (np.std(X, keepdims=True) + tf.keras.backend.epsilon())
        X = np.array(X, dtype=np.float32)
        mean = np.mean(X, axis=0)
        std = np.std(X, axis=0)

        X = (X - mean)/std

        # print(len(X[0]))
        # print(len(X[0][0]))

        # min_max_scaler = preprocessing.MinMaxScaler()
        
        Y = np.array(Y, dtype=int)
        Y1 = np.array(Y1, dtype=int)
        return X, Y, Y1




## Contant

#### Classify

In [ ]:
# Set the config for training

BATCH_SIZE = 32
# LIST_LABELS = ['normal', 'deep', 'heavy', 'other']
LIST_LABELS = ['normal', 'deep', 'heavy', 'other']
N_CLASSES = len(LIST_LABELS)
N_EPOCHS = 100

# INPUT_SIZE = (40, 140, 1) # Input size for CNN training
# INPUT_SIZE = (40, 128) # Input size for LSTM training
# INPUT_SIZE = (32, 128, 1) # Input size for CNN training
INPUT_SIZE = (32, 128) # Input size for LSTM training


TRAINING_SOURCE = '/content/data/public/training/'
VALID_SOURCE = '/content/data-public/data-validation/'
MODE = 'TRAINING'
MODEL_OUTPUT = '/content/drive/My Drive/Breath-Data/data/model_output'

RUN_TITLE = "20210805-"

In [ ]:
# !cp -r '/content/gdrive/My Drive/Breath-Data/data-public/' '/content/'

# Set the config for training

BATCH_SIZE = 32
# LIST_LABELS = ['normal', 'deep', 'heavy', 'other']
# LIST_LABELS = ['Bronchiolitis', 'Bronchiectasis', 'LRTI', 'Asthma', 'URTI', 'Pneumonia', 'Healthy', 'COPD']
# ['Bronchiolitis', 'Bronchiectasis', 'LRTI', 'Asthma', 'URTI', 'Pneumonia', 'Healthy', 'COPD']
LIST_LABELS = ['crackle', 'normal', 'wheeze']
# LIST_LABELS = ['abnormal', 'normal']
N_CLASSES = len(LIST_LABELS)
N_EPOCHS = 20

# INPUT_SIZE = (40, 126, 1) # Input size for CNN training
# INPUT_SIZE = (40, 126) # Input size for LSTM training
# INPUT_SIZE = (32, 140, 1) # Input size for CNN training
INPUT_SIZE = (32, 251) # Input size for LSTM training

TRAINING_SOURCE = '/content/data/public/training/'
VALID_SOURCE = '/content/data-public/data-validation/'
# TRAINING_SOURCE = '/content/gdrive/My Drive/Breath-Data/data-public/'
# VALID_SOURCE = '/content/gdrive/My Drive/Breath-Data/data-public/data-validation/'
MODE = 'TRAINING'
MODEL_OUTPUT = '/content/gdrive/My Drive/Breath-Data/data/model_output'

RUN_TITLE = "data-public"

#### Detection

In [ ]:
# Set the config for training

BATCH_SIZE = 64
# LIST_LABELS = ['normal', 'deep', 'heavy', 'other']
LIST_LABELS = ['normal', 'abnormal']
N_CLASSES = len(LIST_LABELS)
N_EPOCHS = 10


# INPUT_SIZE = (32, 140, 1) # Input size for CNN training
INPUT_SIZE = (32, 251) # Input size for LSTM training


TRAINING_SOURCE = '/content/data/public/training/'
VALID_SOURCE = '/content/data-public/data-validation/'
MODE = 'TRAINING'
MODEL_OUTPUT = '/content/gdrive/My Drive/Breath-Data/data/model_output'

RUN_TITLE = "26082021-Residual"

#####################################
# Set the config for training

# BATCH_SIZE1 = 64
# # LIST_LABELS = ['normal', 'deep', 'heavy', 'other']
# LIST_LABELS1 = ['normal', 'deep', 'heavy', 'other']
# N_CLASSES1 = len(LIST_LABELS1)
# N_EPOCHS1 = 10


# # INPUT_SIZE = (32, 140, 1) # Input size for CNN training
# INPUT_SIZE1 = (32, 251) # Input size for LSTM training


# TRAINING_SOURCE1 = '/content/gdrive/My Drive/Breath-Data/data/training/'
# VALID_SOURCE1 = '/content/gdrive/My Drive/Breath-Data/data/validation/'
# MODE1 = 'TRAINING'
# MODEL_OUTPUT1 = '/content/gdrive/My Drive/Breath-Data/data/model_output'

# RUN_TITLE1 = "20210706-Residual"

## Get Data

In [ ]:
# Generate data for training
# try:
train_generator = BreathDataTrainingGenerator(
          TRAINING_SOURCE,
          list_labels=LIST_LABELS,
          batch_size=BATCH_SIZE,
          dim=INPUT_SIZE,
          shuffle=False)
# except Exception as e:
#   pass

N_TRAIN_SAMPLES = len(train_generator.wavs)
print("Train samples: {}".format(N_TRAIN_SAMPLES))

validation_generator = BreathDataTrainingGenerator(
        VALID_SOURCE,
        list_labels=LIST_LABELS,
        batch_size=BATCH_SIZE,
        dim=INPUT_SIZE,
        shuffle=False)
N_VALID_SAMPLES = len(validation_generator.wavs)
print("Validation samples: {}".format(N_VALID_SAMPLES))

##################################################
# Generate data for training
# try:
# train_generator1 = BreathDataTrainingGenerator(
#           TRAINING_SOURCE1,
#           list_labels=LIST_LABELS1,
#           batch_size=BATCH_SIZE1,
#           dim=INPUT_SIZE1,
#           shuffle=False)
# # except Exception as e:
# #   pass

# N_TRAIN_SAMPLES1 = len(train_generator1.wavs)
# print("Train samples: {}".format(N_TRAIN_SAMPLES1))

# validation_generator1 = BreathDataTrainingGenerator(
#         VALID_SOURCE1,
#         list_labels=LIST_LABELS1,
#         batch_size=BATCH_SIZE1,
#         dim=INPUT_SIZE1,
#         shuffle=False)
# N_VALID_SAMPLES1 = len(validation_generator1.wavs)
# print("Validation samples: {}".format(N_VALID_SAMPLES1))

## 1) CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K

def get_recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    # f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# import tensorflow_addons as tfa
class CNN_MODEL(object):
    @staticmethod
    def build(input_shape, classes, learning_rate):
        # model = Sequential()
        # model.add(CuLSTM(units=128, return_sequences=True, input_shape=data_input_shape))
        # model.add(CuLSTM(units=128,  return_sequences=False))
        # model.add(Dense(units=64, activation="relu"))
        # model.add(Dense(units=classes, activation="softmax"))
        # # Keras optimizer defaults:
        # # Adam   : lr=0.001, beta_1=0.9,  beta_2=0.999, epsilon=1e-8, decay=0.
        # # RMSprop: lr=0.001, rho=0.9,                   epsilon=1e-8, decay=0.
        # # SGD    : lr=0.01,  momentum=0.,                             decay=0.
        # opt = Adam(lr=learning_rate)
        # model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
        # return model


        model = Sequential()

        model.add(Conv2D(80, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform',
                      input_shape=input_shape))
        model.add(LeakyReLU())
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))

        model.add(Conv2D(160, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform'))
        model.add(LeakyReLU())
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))

        model.add(Conv2D(240, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform'))
        model.add(LeakyReLU())
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))

        model.add(Flatten())
        model.add(Dropout(0.5))

        model.add(Dense(classes, kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform'))
        model.add(Activation('softmax'))

        optimizer = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1])
      
        return model

In [ ]:
model = CNN_MODEL.build(input_shape=INPUT_SIZE, classes=N_CLASSES, learning_rate=0.001)

model.summary()

# Checkpoint
if not os.path.exists(MODEL_OUTPUT):
    os.makedirs(MODEL_OUTPUT)

filepath= os.path.join(MODEL_OUTPUT, RUN_TITLE + "CNN-model-{epoch:02d}-{accuracy:.2f}.hdf5") 
# filepath="./model_output/LSTM-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# Start training
history = model.fit_generator(
        train_generator,
        steps_per_epoch= N_TRAIN_SAMPLES // BATCH_SIZE,
        initial_epoch=0,
        epochs=N_EPOCHS,
        validation_data=validation_generator,
        validation_steps=N_VALID_SAMPLES // BATCH_SIZE,
        callbacks=callbacks_list,
        use_multiprocessing=True,
)

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/Breath-Data/cnn-breath-accuracy-mfcc-rnnoise-12082021.pdf')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/Breath-Data/cnn-breath-loss-mfcc-rnnoise-12082021.pdf')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

n_batches = len(validation_generator)

confusion_matrix(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(n_batches)]),    
    np.argmax(model.predict_generator(validation_generator, steps=n_batches), axis=1) 
)



In [ ]:
from sklearn.metrics import classification_report

n_batches = len(validation_generator)

classification_report(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(n_batches)]),    
    np.argmax(model.predict_generator(validation_generator, steps=n_batches), axis=1) 
)


## 2) BiLSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K

def get_recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    # f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

class LSTM_MODEL(object):
    @staticmethod
    def build_simple_lstm(data_input_shape, classes, learning_rate):
        model = Sequential()
        model.add(CuLSTM(units=128, return_sequences=True, input_shape=data_input_shape))
        model.add(CuLSTM(units=128,  return_sequences=False))
        model.add(Dense(units=64, activation="relu"))
        model.add(Dense(units=classes, activation="softmax"))
        # Keras optimizer defaults:
        # Adam   : lr=0.001, beta_1=0.9,  beta_2=0.999, epsilon=1e-8, decay=0.
        # RMSprop: lr=0.001, rho=0.9,                   epsilon=1e-8, decay=0.
        # SGD    : lr=0.01,  momentum=0.,                             decay=0.
        opt = Adam(lr=learning_rate)
        model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), get_f1])
        return model
    
    @staticmethod
    def build_bilstm(data_input_shape, classes, learning_rate):
        model = Sequential()
        model.add(Bidirectional(CuLSTM(128), input_shape=data_input_shape))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation='softmax'))
        opt = Adam(lr=learning_rate)
        model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), get_f1])
        # model.summary()

        return model

    @staticmethod
    def build_residual_bilstm(data_input_shape, classes, learning_rate):
        inp = Input(shape=data_input_shape)
        # inp = TimeDistributed(Sequential())(inp)

        # inp = SincConvFast_1(64, 251, sample_frequency)(inp)
        # inp = normalize(inp)
        # inp = LayerNormalization()(inp)
        # inp = SincConvFast(64, 251, sample_frequency)(inp)
        z1 = Bidirectional(CuLSTM(64, return_sequences=True))(inp)
        z2 = Bidirectional(CuLSTM(units=64, return_sequences=True))(z1)
        z3 = add([z1, z2])  # residual connection
        z4 = Bidirectional(CuLSTM(units=64, return_sequences=True))(z3)
        z5 = Bidirectional(CuLSTM(units=64, return_sequences=False))(z4)
        z6 = add([z4, z5])  # residual connection    
        z61 = Flatten()(z6)        
        z7 = Dense(128, activation='relu')(z61)
        z8 = Dropout(0.5)(z7)
        # out = Dense(classes, activation='softmax')(z8)
        out = Dense(classes, activation='softmax')(z8)
        model = Model(inputs=[inp], outputs=out)
        opt = Adam(lr=learning_rate)
        model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1])
        # model.summary()
        return model

In [ ]:
# INPUT_SIZE = (32, 251) # Input size for BiLSTM training

model = LSTM_MODEL.build_residual_bilstm(data_input_shape=INPUT_SIZE, classes=N_CLASSES, learning_rate=0.001)

filepath= os.path.join(MODEL_OUTPUT, RUN_TITLE + "bi_lstm-{epoch:02d}-{accuracy:.2f}-{val_accuracy:.2f}.hdf5") 
# filepath="./model_output/LSTM-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=False, mode='max')
callbacks_list = [checkpoint]



# Start training
history = model.fit_generator(
        train_generator,
        steps_per_epoch= N_TRAIN_SAMPLES // BATCH_SIZE,
        initial_epoch=0,
        epochs=N_EPOCHS,
        validation_data=validation_generator,
        validation_steps=N_VALID_SAMPLES // BATCH_SIZE,
        callbacks=callbacks_list,
        use_multiprocessing=True,
)

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/Breath-Data/bilstm-breath-accuracy-mfcc-14082021.pdf')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/Breath-Data/bilstm-breath-loss-mfcc-13082021.pdf')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

n_batches = len(validation_generator)

confusion_matrix(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(n_batches)]),    
    np.argmax(model.predict_generator(validation_generator, steps=n_batches), axis=1) 
)


In [ ]:
from sklearn.metrics import classification_report

n_batches = len(validation_generator)

classification_report(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(n_batches)]),    
    np.argmax(model.predict_generator(validation_generator, steps=n_batches), axis=1) 
)


## 3) LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import GlobalAveragePooling1D

import tensorflow as tf
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K

def get_recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    # f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


# https://github.com/lvrysis/Audio-DNN-Classification/blob/master/Audio-DNN/Python/DeepCNN2DLSTM.py

class LSTM_MODEL(object):
    @staticmethod
    def build_simple_lstm(data_input_shape, classes, learning_rate):
        model = Sequential()
        model.add(CuLSTM(units=128, return_sequences=True, input_shape=data_input_shape))
        model.add(CuLSTM(units=128,  return_sequences=False))
        model.add(Dense(units=64, activation="relu"))
        model.add(Dense(units=classes, activation="softmax"))
        # Keras optimizer defaults:
        # Adam   : lr=0.001, beta_1=0.9,  beta_2=0.999, epsilon=1e-8, decay=0.
        # RMSprop: lr=0.001, rho=0.9,                   epsilon=1e-8, decay=0.
        # SGD    : lr=0.01,  momentum=0.,                             decay=0.
        opt = Adam(lr=learning_rate)
        model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1])

        model.summary()
        return model

In [ ]:
# INPUT_SIZE_2 = (32, 251, 1) # Input size for CNN training

model = LSTM_MODEL.build_simple_lstm(data_input_shape=INPUT_SIZE, classes=N_CLASSES, learning_rate=0.001)

filepath= os.path.join(MODEL_OUTPUT, RUN_TITLE + "LSTM-weights-improvement_bi_lstm-{epoch:02d}-{accuracy:.2f}-{val_accuracy:.2f}.hdf5")
# filepath="./model_output/LSTM-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# Start training
model.fit_generator(
        train_generator,
        steps_per_epoch= N_TRAIN_SAMPLES // BATCH_SIZE,
        initial_epoch=0,
        epochs=N_EPOCHS,
        validation_data=validation_generator,
        validation_steps=N_VALID_SAMPLES // BATCH_SIZE,
        callbacks=callbacks_list,
        # use_multiprocessing=True,
)

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/Breath-Data/lstm-public-accuracy-mfcc-rnnoise-12082021.pdf')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/Breath-Data/lstm-public-loss-mfcc-rnnoise-12082021.pdf')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

n_batches = len(validation_generator)

confusion_matrix(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(n_batches)]),    
    np.argmax(model.predict_generator(validation_generator, steps=n_batches), axis=1) 
)


In [ ]:
from sklearn.metrics import classification_report

n_batches = len(validation_generator)

classification_report(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(n_batches)]),    
    np.argmax(model.predict_generator(validation_generator, steps=n_batches), axis=1) 
)

In [ ]:
# Number of steps corresponding to an epoch
steps = 10
predictions = model.predict_generator(validation_generator, steps=steps)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(
    np.concatenate([np.argmax(validation_generator[i][1], axis=1) for i in range(300)]),    
    np.argmax(model.predict_generator(validation_generator, steps=300), axis=1) 
)


In [ ]:
# label names
labels = validation_generator.class_indices.keys()
precisions, recall, f1_score, _ = metrics.precision_recall_fscore_support(val_trues, val_preds, labels=labels)

In [ ]:
print(precisions)
print(recall)
print(f1_score)

## 5) Multi-task - BiLSTM or CNN: detect + classify

### Get Data Multi Output

In [ ]:
# Generate data for training
# try:
train_generator_multi = BreathDataTrainingGeneratorMulti(
          TRAINING_SOURCE,
          list_labels=LIST_LABELS,
          batch_size=BATCH_SIZE,
          dim=INPUT_SIZE,
          shuffle=False)
# except Exception as e:
#   pass

N_TRAIN_SAMPLES = len(train_generator_multi.wavs)
print("Train samples: {}".format(N_TRAIN_SAMPLES))

validation_generator_multi = BreathDataTrainingGeneratorMulti(
        VALID_SOURCE,
        list_labels=LIST_LABELS,
        batch_size=BATCH_SIZE,
        dim=INPUT_SIZE,
        shuffle=False)
N_VALID_SAMPLES = len(validation_generator_multi.wavs)
print("Validation samples: {}".format(N_VALID_SAMPLES))

### Init Model

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import GlobalAveragePooling1D

import tensorflow as tf
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K

def get_recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    # f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def get_trainable_model_bilstm(data_input_shape, classes, learning_rate):
    inp1 = Input(shape=data_input_shape) # detect
    inp2 = Input(shape=data_input_shape) # classify

    z1 = Bidirectional(CuLSTM(128, return_sequences=True))(inp1)
    z2 = Bidirectional(CuLSTM(units=128, return_sequences=True))(z1)
    z3 = add([z1, z2])  # residual connection
    z4 = Bidirectional(CuLSTM(units=128, return_sequences=True))(z3)
    z5 = Bidirectional(CuLSTM(units=128, return_sequences=False))(z4)
    z6 = add([z4, z5])  # residual connection    
    z61 = Flatten()(z6) 
    z7 = Dense(256, activation='relu')(z61)
    z8 = Dropout(0.5)(z7)
    # out = Dense(classes, activation='softmax')(z8)
    out1 = Dense(3, activation='softmax', name='output_1')(z8)
    out2 = Dense(2, activation='softmax', name='output_2')(z8)

    model = Model(inputs=inp1, outputs=[out1, out2])
    opt = Adam(lr=learning_rate)
    losses = {
          "output_1": "categorical_crossentropy",
          "output_2": "categorical_crossentropy"
    }
    metricss = {
        "output_1": ['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1],
        "output_2": ['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1]
    }
    model.compile(loss=losses, optimizer=opt, metrics=metricss)
    model.summary()
    return model

In [ ]:
import tensorflow

def get_trainable_model_cnn(data_input_shape, classes, learning_rate):
    inp = Input(shape=data_input_shape) # detect

    c1 = Conv2D(80, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform')(inp)
    c2 = LeakyReLU()(c1)
    c3 = MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same')(c2)

    c4 = Conv2D(160, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform')(c3)
    c5 = LeakyReLU()(c4)
    c6 = MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same')(c5)

    c7 = Conv2D(240, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform')(c6)
    c8 = LeakyReLU()(c7)
    c9 = MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same')(c8)

    c10 = Flatten()(c9)
    c11 = Dropout(0.5)(c10)

    out1 = Dense(4, activation='softmax', name='output_1')(c11)
    out2 = Dense(2, activation='softmax', name='output_2')(c11)

    model = Model(inputs=inp, outputs=[out1, out2])
    opt = Adam(lr=learning_rate)
    losses = {
          "output_1": "categorical_crossentropy",
          "output_2": "categorical_crossentropy"
    }
    metricss = {
        "output_1": ['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1],
        "output_2": ['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1]
    }
    model.compile(loss=losses, optimizer=opt, metrics=metricss)
    model.summary()
    return model

In [ ]:
import tensorflow

def get_trainable_model_lstm(data_input_shape, classes, learning_rate):
    inp = Input(shape=data_input_shape)

    l1 = CuLSTM(units=128, return_sequences=True)(inp)
    l2 = CuLSTM(units=128,  return_sequences=False)(l1)
    l3 = Dense(units=64, activation="relu")(l2)

    out1 = Dense(4, activation='softmax', name='output_1')(l3)
    out2 = Dense(2, activation='softmax', name='output_2')(l3)

    model = Model(inputs=inp, outputs=[out1, out2])
    opt = Adam(lr=learning_rate)
    losses = {
          "output_1": "categorical_crossentropy",
          "output_2": "categorical_crossentropy"
    }
    metricss = {
        "output_1": ['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1],
        "output_2": ['accuracy', tf.keras.metrics.Precision(), get_recall, get_f1]
    }
    model.compile(loss=losses, optimizer=opt, metrics=metricss)
    model.summary()
    return model

In [ ]:
model = get_trainable_model_bilstm(data_input_shape=INPUT_SIZE, classes=N_CLASSES, learning_rate=0.001)
# model = get_trainable_model_cnn(data_input_shape=INPUT_SIZE, classes=N_CLASSES, learning_rate=0.001)
# model = get_trainable_model_lstm(data_input_shape=INPUT_SIZE, classes=N_CLASSES, learning_rate=0.001)

filepath= os.path.join(MODEL_OUTPUT, RUN_TITLE + "bi_lstm-{epoch:02d}.hdf5")
# filepath="./model_output/LSTM-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# Start training
model.fit_generator(
        train_generator_multi,
        steps_per_epoch= N_TRAIN_SAMPLES // BATCH_SIZE,
        initial_epoch=0,
        epochs=N_EPOCHS,
        validation_data=validation_generator_multi,
        validation_steps=N_VALID_SAMPLES // BATCH_SIZE,
        callbacks=callbacks_list,
        # use_multiprocessing=True,
)